In [ ]:
# spark.conf.set("spark.hadoop.google.cloud.auth.service.account.enable", "true")
# spark.conf.set("spark.hadoop.fs.gs.auth.service.account.email", "bucket-bigquery@leafy-environs-409823.iam.gserviceaccount.com")
# spark.conf.set("spark.hadoop.fs.gs.project.id", "leafy-environs-409823")
# spark.conf.set("spark.hadoop.fs.gs.auth.service.account.private.key", dbutils.secrets.get(scope='gcp-bucket', key='databricks-bucket-key'))
# spark.conf.set("spark.hadoop.fs.gs.auth.service.account.private.key.id", dbutils.secrets.get(scope='gcp-bucket', key='databricks-bucket-key_id'))

In [ ]:
from pyspark.sql.functions import col, when, regexp_replace, lit
from pathlib import Path

In [ ]:
# GCP secret -> steps to install
# databricks configure --token
# databricks secrets create-scope gcp-bucket --initial-manage-principal users
# databricks secrets put-secret gcp-bucket databricks-bucket-key
# databricks secrets put-secret gcp-bucket databricks-bucket-key_id
# Add config on Cluster Spark config https://docs.gcp.databricks.com/en/connect/storage/gcs.html

print(dbutils.secrets.listScopes())
print(dbutils.secrets.list('gcp-bucket'))

dbutils.fs.ls("gs://bossa-bucket-coutj/")


In [ ]:

#READING datasus raw data parquet

dbutils.widgets.text("file_name", "", "Enter file_name")
file_name = dbutils.widgets.get('file_name') or '2016/folha_2016_1.parquet'
# print(file_name)
df = spark.read.parquet(f"gs://bossa-bucket-coutj/raw/{file_name}")

In [ ]:
# Remove "-"

for column in df.columns:
    # print(column)
    df = df.withColumn(column, when( col(column)=='-', None).otherwise(col(column)))
    
# display(df)

In [ ]:
# Fix currency format and cast string values to float
numeric_columns = [col for col in df.columns if col not in ['nome', 'cargo', 'funcao']]

for column in numeric_columns:
    df = df.withColumn(
                column, regexp_replace(col(column), "\.", "")
                ).withColumn(column, regexp_replace(column, ",", ".").cast("float"))


In [ ]:
# Fix column names

for column in df.columns:
    df = df.withColumnRenamed(column, column.replace("__", "_"))

In [ ]:
# Define data date
import re
from datetime import datetime
date = re.search(r"(?<=folha_)(.*)(?=\.parquet)",file_name).group(0).split('_')
date = datetime.strptime(f"{date[0]}/{date[1]}/1","%Y/%m/%d")
table_name = f"{date[0]}_{date[1]}"
print(date)

In [ ]:
# Add date column
df = df.withColumn('date',lit(date))
# display(df)

In [ ]:
# import com.google.cloud.spark.bigquery.BigQueryDataFrameReader
# import sqlContext.implicits._

# Write dataframe to Bigquery:
# adiciona project talvez? tenta ai
# bossabyte.alerj
#.option("credentials", dbutils.secrets.get('gcp-bucket', 'service-secret'))


(df.write.format("bigquery")
    .mode("overwrite")
    .option("project", 'leafy-environs-409823')
    .option("parentProject", 'leafy-environs-409823')
    .option("temporaryGcsBucket","bossa-bucket-coutj")
    .option("table",f"leafy-environs-409823.alerj_ds.alerj_payslip_{}")
    .save())

In [ ]:
df.write.mode('overwrite').parquet(f'gs://bossa-bucket-coutj/trusted/{file_name}')